In [ ]:
import pandas as pd

In [ ]:
pd.__version__

In [ ]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df.head()

In [ ]:
df_copy = df.copy()

In [ ]:
df_copy.info()

In [ ]:
df_copy['duration'] = (df_copy['tpep_dropoff_datetime'] - df_copy['tpep_pickup_datetime']).dt.total_seconds() / 60

In [ ]:
df_copy.describe()

In [ ]:
# duration std
4.259435e+01

In [ ]:
df_copy = df_copy.loc[(df_copy['duration'] >= 1) & (df_copy['duration'] <= 60)]

In [ ]:
# size left after dropping outliers in `duration` col
(len(df_copy) / len(df)) * 100 

In [ ]:
df_train = df_copy[['PULocationID', 'DOLocationID']]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder(drop='first', handle_unknown='ignore')
df_train_encoded = encoder.fit_transform(df_train)

In [ ]:
df_train_encoded.shape

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X, y = df_train_encoded, df_copy['duration'].values

In [ ]:
lr_model = LinearRegression().fit(X, y)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
y_pred = lr_model.predict(X)
mean_squared_error(y, y_pred, squared=False)

In [ ]:
def val_pipeline(data_val_url, encoder, model):
	df_val = pd.read_parquet(data_val_url)
	df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60
	df_val = df_val.loc[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]

	y_val = df_val['duration'].values
	df_val = df_val[['PULocationID', 'DOLocationID']]
	X_val = encoder.transform(df_val)
	
	y_pred_val = model.predict(X_val)
	
	return mean_squared_error(y_val, y_pred_val, squared=False)

In [ ]:
data_val_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'
val_pipeline(data_val_url, encoder, lr_model)